In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
import pickle
from sklearn import preprocessing
from sklearn.manifold import SpectralEmbedding
from sklearn.model_selection import StratifiedKFold

Get the Data - The csv is very large, so we need to assign data types to control the memory required.

In [ ]:
train_df =pd.read_pickle('../input/ump-train-picklefile/train.pkl')
#train_df.info(verbose = True) # dataset is too large for pandas profiling (e.g. auto EDA tool)
train_df = train_df.reset_index(drop = False)
y_time = train_df.pop('time_id').values
y = train_df.pop('target').values

In [ ]:
train_df.pop('row_id')

Prepare the Data - Scale each feature of x to have a mean of 0 and standard deviation of 1. 
                   Reduce the dimensionality using PCA 

In [ ]:
x = train_df.values
# scaler = preprocessing.StandardScaler().fit(features)
# pickle.dump(scaler, open('scaler.pkl','wb'))
# features = scaler.transform(features)

# pca = PCA(n_components=150, random_state=22)
# pca.fit(features)
# x = pca.transform(features)
# pickle.dump(pca,  open('pca.pkl', 'wb'))

Train the base models - Using Stratified Kfolding creates multiple datasets with sample distributions equal to the overall training dataset. Each fold will train 1 model. A final regressor will then ensemble the models' using their predictions as training data.


In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

i = 0
models = []
skf = StratifiedKFold(n_splits=30)

for train_index, test_index in skf.split(x, y_time):
    regr = LinearRegression()
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    regr.fit(X_train, y_train)
    pickle.dump(regr,  open('trained_lr'+str(i)+'.pkl', 'wb'))
    models.append(regr)
    i += 1

Train final meta model - This model takes the predictions from the other models as input and produces a final prediction

In [ ]:
df = pd.DataFrame(columns=np.arange(len(models)))

for i, model in enumerate(models):
    df[i]=model.predict(x)

regr = LinearRegression()
regr.fit(df.values, y)

pickle.dump(regr,  open('final_trained_lr.pkl', 'wb'))

Future Work: Use LSTM for time series prediction

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM

# model = Sequential()
# model.add(LSTM(50, input_shape=(x.shape[1], x.shape[2])))
# model.add(Dense(1))
# model.compile(loss = 'mae', optimizer = 'adam')
# model.fit(x, y)

# model.save('trained_lstm')

